In [5]:
import pandas as pd

# Ruta al archivo CSV
archivo_csv = 'Diccionario_nut.csv'

# Cargar el archivo CSV
df = pd.read_csv(archivo_csv, sep=';', header=None)

# Quedarse solo con la primera columna
primera_columna = df[0]

# Filtrar los valores no nulos o no vacíos en la primera columna
primera_columna_limpia = primera_columna.dropna()

# Guardar los resultados en un nuevo archivo CSV
primera_columna_limpia.to_csv('Diccionario_nut_limpio.csv', index=False, header=False)

print("Archivo CSV limpio guardado como 'Diccionario_nut_limpio.csv'")


Archivo CSV limpio guardado como 'Diccionario_nut_limpio.csv'


In [ ]:
import pandas as pd
import numpy as np

# Cargar archivo de gastos (usando un archivo CSV equivalente)
ruta_gastos = 'ENGIH 2016 Base de Datos Gastos.csv' 
df_gastos = pd.read_csv(ruta_gastos, encoding='windows-1252')

# Crear una nueva columna `gasto_hogar` basada en las condiciones (DESTINOCODIGO 1 o 5)
df_gastos['gasto_hogar'] = np.where((df_gastos['DESTINOCODIGO'] == 1) | (df_gastos['DESTINOCODIGO'] == 5), 1, 0)

# Filtrar por gasto_hogar igual a 1
df_gastos_filtrado = df_gastos[df_gastos['gasto_hogar'] == 1]

# Guardar el archivo limpio (opcional, si deseas guardar)
df_gastos_filtrado.to_csv('Base_Gastos_Con_Nutrientes.csv', index=False, encoding='windows-1252')


In [ ]:
# Cargar la tabla de composición química
ruta_composicion = 'Tabla_composicion_para_mach_01_06_2021.csv'
df_composicion = pd.read_csv(ruta_composicion, encoding='windows-1252')

# Hacer el match por 'ARTICULOCODIGO'
df_merged = pd.merge(df_gastos_filtrado, df_composicion, on='ARTICULOCODIGO', how='left')

# Hacer el match con otras tablas
ruta_ninos_hogar = 'variables intermedias/Niños_hogar.csv'
df_ninos_hogar = pd.read_csv(ruta_ninos_hogar, encoding='windows-1252')

df_merged = pd.merge(df_merged, df_ninos_hogar, on='NUMERO', how='left')

# Cargar otras tablas y hacer merge con CCIF_2016
ruta_grupos = 'variables intermedias/Grupos_2006_y_ultra_msp.csv'
df_grupos = pd.read_csv(ruta_grupos, encoding='windows-1252')

df_merged = pd.merge(df_merged, df_grupos, on='CCIF_2016', how='left')


In [ ]:
# Crear o actualizar columnas basadas en condiciones
df_merged['Na_2016'] = np.where(df_merged['CCIF_2016'] == 1121110, 61.15, df_merged['Na_2016'])
df_merged['Na_2016'] = np.where(df_merged['CCIF_2016'] == 1121120, 61.15, df_merged['Na_2016'])
df_merged['Na_2016'] = np.where(df_merged['CCIF_2016'] == 1124050, 82, df_merged['Na_2016'])
df_merged['Na_2016'] = np.where(df_merged['CCIF_2016'].isin([1125041, 1125042, 1125051, 1125052]), 970, df_merged['Na_2016'])

# Calcular nuevas columnas de sal y diferencias
df_merged['sal_2'] = df_merged['Na_2016'] / 400
df_merged['DIF_sal'] = df_merged['sal_2016'] - df_merged['sal_2']


In [ ]:
# Identificar casos duplicados basados en NUMERO
df_merged.sort_values(by='NUMERO', inplace=True)

# Crear un indicador de duplicado
df_merged['PrimarioPrimero'] = df_merged.duplicated(subset='NUMERO', keep='first')
df_merged['hogar'] = np.where(df_merged['PrimarioPrimero'] == False, 1, 0)

In [ ]:
# Recalcular CANTIDADCONVERTIDA y calorías finales
df_merged['CANTIDADCONVERTIDA'] = df_merged['CANTIDADCONVERTIDA_0'] / df_merged['FC']
df_merged['Calorias_final'] = df_merged['CANTIDADCONVERTIDA'] * df_merged['Calorias_2016'] / 100

# Calcular gr_mes basado en la cantidad convertida
df_merged['gr_mes'] = df_merged['CANTIDADCONVERTIDA'] * 4.3

In [ ]:
# Crear la columna de Niños_hogar basado en la suma de Niños_hogar_sum
df_merged['Niños_hogar'] = np.where(df_merged['Niños_hogar_sum'] >= 1, 1, 0)

# Asignar etiquetas a la nueva columna
df_merged['Niños_hogar'] = df_merged['Niños_hogar'].map({0: "Hogar sin niños", 1: "Hogar con niños"})


In [ ]:
# Guardar el archivo final
df_merged.to_csv('C:/Users/Cecilia/Desktop/Base_Gastos_Con_Nutrientes_Final.csv', index=False, encoding='windows-1252')

print("Archivo final guardado.")


In [ ]:
# Cálculo de gramos por día
df_merged['gr_dia'] = df_merged['CANTIDADCONVERTIDA'] * 52.179 / 366

# Agregar por 'NUMERO' para calcular gramos por hogar
df_gr_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_dia'].sum()
df_gr_hogar.rename(columns={'gr_dia': 'gr_hogar'}, inplace=True)

# Calcular gramos per cápita
df_gr_hogar['gr_percapita'] = df_gr_hogar['gr_hogar'] / df_merged['nper_hogar']

# Calcular calorías por artículo por día
df_merged['Cal_articulo_dia'] = df_merged['gr_dia'] * df_merged['Calorias_2016'] / 100

# Agregar por 'NUMERO' para calorías por hogar
df_cal_hogar = df_merged.groupby('NUMERO', as_index=False)['Cal_articulo_dia'].sum()
df_cal_hogar.rename(columns={'Cal_articulo_dia': 'Calorias_hogar'}, inplace=True)

# Calorías per cápita
df_cal_hogar['Cal_percapita'] = df_cal_hogar['Calorias_hogar'] / df_merged['nper_hogar']

# Adecuación calórica
df_cal_hogar['adec_kcal'] = df_cal_hogar['Cal_percapita'] * 100 / df_merged['reqk_mean']


In [ ]:
# Cálculo de proteínas en gramos y su energía en calorías
df_merged['gr_proteinas'] = df_merged['gr_dia'] * df_merged['Proteinas_2016'] / 100
df_merged['gr_proteinas'].fillna(0, inplace=True)

# Agregar proteínas por hogar
df_prot_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_proteinas'].sum()
df_prot_hogar.rename(columns={'gr_proteinas': 'prot_hogar'}, inplace=True)
df_prot_hogar['cal_proteinas'] = df_prot_hogar['prot_hogar'] * 4

# Cálculo de grasas en gramos y su energía en calorías
df_merged['gr_grasas'] = df_merged['gr_dia'] * df_merged['Grasas_2016'] / 100
df_merged['gr_grasas'].fillna(0, inplace=True)

# Agregar grasas por hogar
df_grasas_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_grasas'].sum()
df_grasas_hogar.rename(columns={'gr_grasas': 'grasas_hogar'}, inplace=True)
df_grasas_hogar['cal_grasas'] = df_grasas_hogar['grasas_hogar'] * 9

# Cálculo de hidratos en gramos y su energía en calorías
df_merged['gr_hidratos'] = df_merged['gr_dia'] * df_merged['Hidratos_2016'] / 100
df_merged['gr_hidratos'].fillna(0, inplace=True)

# Agregar hidratos por hogar
df_hidratos_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_hidratos'].sum()
df_hidratos_hogar.rename(columns={'gr_hidratos': 'hidratos_hogar'}, inplace=True)
df_hidratos_hogar['cal_hidratos'] = df_hidratos_hogar['hidratos_hogar'] * 4

# Sumar las calorías de los macronutrientes
df_calorias_macro = pd.merge(df_prot_hogar, df_grasas_hogar, on='NUMERO')
df_calorias_macro = pd.merge(df_calorias_macro, df_hidratos_hogar, on='NUMERO')
df_calorias_macro['calorias_macro'] = df_calorias_macro['cal_proteinas'] + df_calorias_macro['cal_grasas'] + df_calorias_macro['cal_hidratos']

# Adecuación de los macronutrientes
df_calorias_macro['adec_proteinas'] = df_calorias_macro['cal_proteinas'] * 100 / df_calorias_macro['calorias_macro']
df_calorias_macro['adec_grasas'] = df_calorias_macro['cal_grasas'] * 100 / df_calorias_macro['calorias_macro']
df_calorias_macro['adec_hidratos'] = df_calorias_macro['cal_hidratos'] * 100 / df_calorias_macro['calorias_macro']


In [ ]:
# Cálculo de ácidos grasos saturados
df_merged['gr_AGS'] = df_merged['gr_dia'] * df_merged['AGS_2016'] / 100
df_merged['gr_AGS'].fillna(0, inplace=True)

# Agregar ácidos grasos saturados por hogar
df_AGS_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_AGS'].sum()
df_AGS_hogar.rename(columns={'gr_AGS': 'AGS_hogar'}, inplace=True)
df_AGS_hogar['cal_AGS'] = df_AGS_hogar['AGS_hogar'] * 9

# Calorías de ácidos grasos saturados per cápita
df_AGS_hogar['cal_AGS_percapita'] = df_AGS_hogar['cal_AGS'] / df_merged['nper_hogar']
df_AGS_hogar['adec_AGS'] = df_AGS_hogar['cal_AGS_percapita'] * 100 / df_cal_hogar['Cal_percapita']


In [ ]:
# Cálculo de azúcares libres
df_merged['gr_azucar_libre'] = df_merged['gr_hidratos'] * df_merged['prop_azucarlibre_2016'] / 100
df_merged['gr_azucar_libre'].fillna(0, inplace=True)

# Agregar azúcares libres por hogar
df_azu_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_azucar_libre'].sum()
df_azu_hogar.rename(columns={'gr_azucar_libre': 'azu_hogar'}, inplace=True)
df_azu_hogar['cal_azucar_libre'] = df_azu_hogar['azu_hogar'] * 4

# Calorías de azúcares libres per cápita
df_azu_hogar['azu_libre_per_capita'] = df_azu_hogar['cal_azucar_libre'] / df_merged['nper_hogar']
df_azu_hogar['prop_azucar_libre'] = df_azu_hogar['azu_libre_per_capita'] * 100 / df_cal_hogar['Cal_percapita']



In [ ]:
# Cálculo de sal
df_merged['sal_dia'] = df_merged['gr_dia'] * df_merged['sal_2016'] / 100
df_merged['sal_dia'].fillna(0, inplace=True)

# Agregar sal por hogar
df_sal_hogar = df_merged.groupby('NUMERO', as_index=False)['sal_dia'].sum()
df_sal_hogar.rename(columns={'sal_dia': 'sal_gr0'}, inplace=True)
df_sal_hogar['sal_gr'] = df_sal_hogar['sal_gr0'] / df_merged['nper_hogar']

# Cálculo de sodio
df_merged['sodio_dia'] = df_merged['gr_dia'] * df_merged['Na_2016'] / 100
df_merged['sodio_dia'].fillna(0, inplace=True)
df_sodio_hogar = df_merged.groupby('NUMERO', as_index=False)['sodio_dia'].sum()
df_sodio_hogar.rename(columns={'sodio_dia': 'sodio_mg0'}, inplace=True)
df_sodio_hogar['sodio_mg'] = df_sodio_hogar['sodio_mg0'] / df_merged['nper_hogar']


In [ ]:

# Cálculo de fibra
df_merged['fibra_dia'] = df_merged['gr_dia'] * df_merged['Fibra_2016'] / 100
df_merged['fibra_dia'].fillna(0, inplace=True)
df_fibra_hogar = df_merged.groupby('NUMERO', as_index=False)['fibra_dia'].sum()
df_fibra_hogar.rename(columns={'fibra_dia': 'fibra_gr0'}, inplace=True)
df_fibra_hogar['fibra_gr'] = df_fibra_hogar['fibra_gr0'] / df_merged['nper_hogar']

# Cálculo de hierro
df_merged['hierro_dia'] = df_merged['gr_dia'] * df_merged['FE_2016'] / 100
df_merged['hierro_dia'].fillna(0, inplace=True)
df_hierro_hogar = df_merged.groupby('NUMERO', as_index=False)['hierro_dia'].sum()
df_hierro_hogar.rename(columns={'hierro_dia': 'hierro_mg0'}, inplace=True)
df_hierro_hogar['hierro_percapita'] = df_hierro_hogar['hierro_mg0'] / df_merged['nper_hogar']

# Cálculo de potasio
df_merged['K_dia'] = df_merged['gr_dia'] * df_merged['K_2016'] / 100
df_merged['K_dia'].fillna(0, inplace=True)
df_K_hogar = df_merged.groupby('NUMERO', as_index=False)['K_dia'].sum()
df_K_hogar.rename(columns={'K_dia': 'K_mg0'}, inplace=True)
df_K_hogar['K_percapita'] = df_K_hogar['K_mg0'] / df_merged['nper_hogar']

# Cálculo de ácido fólico
df_merged['AF_dia'] = df_merged['gr_dia'] * df_merged['A_Folico_2016'] / 100
df_merged['AF_dia'].fillna(0, inplace=True)
df_AF_hogar = df_merged.groupby('NUMERO', as_index=False)['AF_dia'].sum()
df_AF_hogar.rename(columns={'AF_dia': 'AF_mg0'}, inplace=True)
df_AF_hogar['AF_percapita'] = df_AF_hogar['AF_mg0'] / df_merged['nper_hogar']

# Cálculo de calcio
df_merged['calcio_dia'] = df_merged['gr_dia'] * df_merged['Ca_2016'] / 100
df_merged['calcio_dia'].fillna(0, inplace=True)
df_calcio_hogar = df_merged.groupby('NUMERO', as_index=False)['calcio_dia'].sum()
df_calcio_hogar.rename(columns={'calcio_dia': 'CA_mg0'}, inplace=True)
df_calcio_hogar['CA_percapita'] = df_calcio_hogar['CA_mg0'] / df_merged['nper_hogar']


In [ ]:
# Calcular grupo de verduras y frutas (grupo_vegyfrutas_2016)
condiciones_vegyfrutas = df_merged['CCIF_2016'].isin([11112430, 11112443])
df_merged['grupo_vegyfrutas_2016'] = np.where(condiciones_vegyfrutas, 1, 0)

# Gramos de verduras y frutas por día
df_merged['vegyfrutagr'] = np.where(df_merged['grupo_vegyfrutas_2016'] == 1, df_merged['gr_dia'], 0)
df_merged['vegyfrutagr'].fillna(0, inplace=True)

# Agregar gramos de verduras y frutas por hogar
df_vegyfrutas_hogar = df_merged.groupby('NUMERO', as_index=False)['vegyfrutagr'].sum()
df_vegyfrutas_hogar.rename(columns={'vegyfrutagr': 'gr_vegyfrutasSUM'}, inplace=True)
df_vegyfrutas_hogar['gr_vegyfruta_percapita'] = df_vegyfrutas_hogar['gr_vegyfrutasSUM'] / df_merged['nper_hogar']


In [ ]:
# Alimentos
df_merged['alimentosgr'] = np.where(df_merged['Alimentos'] == 1, df_merged['gr_dia'], 0)
df_merged['alimentoscal'] = np.where(df_merged['Alimentos'] == 1, df_merged['Cal_articulo_dia'], 0)

# Sumar gramos y calorías de alimentos por hogar
df_aggregated_alimentos = df_merged.groupby('NUMERO', as_index=False).agg({
    'alimentosgr': 'sum',
    'alimentoscal': 'sum'
})

# Gramos y calorías de alimentos per cápita
df_aggregated_alimentos['gr_Alimentos_percapita'] = df_aggregated_alimentos['alimentosgr'] / df_merged['nper_hogar']
df_aggregated_alimentos['cal_Alimentos_percapita'] = df_aggregated_alimentos['alimentoscal'] / df_merged['nper_hogar']


In [ ]:
# Bebidas
df_merged['bebidasgr'] = np.where(df_merged['Alimentos'] == 2, df_merged['gr_dia'], 0)
df_merged['bebidascal'] = np.where(df_merged['Alimentos'] == 2, df_merged['Cal_articulo_dia'], 0)

# Sumar gramos y calorías de bebidas por hogar
df_aggregated_bebidas = df_merged.groupby('NUMERO', as_index=False).agg({
    'bebidasgr': 'sum',
    'bebidascal': 'sum'
})

# Gramos y calorías de bebidas per cápita
df_aggregated_bebidas['gr_bebidas_percapita'] = df_aggregated_bebidas['bebidasgr'] / df_merged['nper_hogar']
df_aggregated_bebidas['cal_bebidas_percapita'] = df_aggregated_bebidas['bebidascal'] / df_merged['nper_hogar']


In [ ]:
# Eliminar variables temporales de bebidas y alimentos
df_merged.drop(columns=['bebidasgr', 'bebidascal', 'alimentosgr', 'alimentoscal'], inplace=True)


In [ ]:
# Panes, galletas, Alfajores y productos de pastelería
df_merged['gr_panes'] = np.where(df_merged['grupos_2006'] == 1, df_merged['gr_dia'], 0)
df_merged['cal_panes'] = np.where(df_merged['grupos_2006'] == 1, df_merged['Cal_articulo_dia'], 0)

# Sumar gramos y calorías de panes por hogar
df_aggregated_panes = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_panes': 'sum',
    'cal_panes': 'sum'
})

# Gramos y calorías de panes per cápita
df_aggregated_panes['gr_panes_percapita'] = df_aggregated_panes['gr_panes'] / df_merged['nper_hogar']
df_aggregated_panes['cal_panes_percapita'] = df_aggregated_panes['cal_panes'] / df_merged['nper_hogar']

# Eliminar variables temporales de panes
df_merged.drop(columns=['gr_panes', 'cal_panes'], inplace=True)


In [ ]:
# Arroz, fideos, harinas y otros cereales
df_merged['gr_Cereales'] = np.where(df_merged['grupos_2006'] == 2, df_merged['gr_dia'], 0)
df_merged['cal_Cereales'] = np.where(df_merged['grupos_2006'] == 2, df_merged['Cal_articulo_dia'], 0)

# Sumar gramos y calorías de cereales por hogar
df_aggregated_cereales = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Cereales': 'sum',
    'cal_Cereales': 'sum'
})

# Gramos y calorías de cereales per cápita
df_aggregated_cereales['gr_Cereales_percapita'] = df_aggregated_cereales['gr_Cereales'] / df_merged['nper_hogar']
df_aggregated_cereales['cal_Cereales_percapita'] = df_aggregated_cereales['cal_Cereales'] / df_merged['nper_hogar']

# Eliminar variables temporales de cereales
df_merged.drop(columns=['gr_Cereales', 'cal_Cereales'], inplace=True)


In [ ]:
# Carnes
df_merged['gr_Carnes'] = np.where(df_merged['grupos_2006'] == 3, df_merged['gr_dia'], 0)
df_merged['cal_Carnes'] = np.where(df_merged['grupos_2006'] == 3, df_merged['Cal_articulo_dia'], 0)

df_aggregated_carnes = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Carnes': 'sum',
    'cal_Carnes': 'sum'
})

df_aggregated_carnes['gr_Carnes_percapita'] = df_aggregated_carnes['gr_Carnes'] / df_merged['nper_hogar']
df_aggregated_carnes['cal_Carnes_percapita'] = df_aggregated_carnes['cal_Carnes'] / df_merged['nper_hogar']

df_merged.drop(columns=['gr_Carnes', 'cal_Carnes'], inplace=True)


In [ ]:
# Mostrar frecuencia de per cápita para grupos de alimentos
df_aggregated_panes[['gr_panes_percapita', 'cal_panes_percapita']].describe()
df_aggregated_cereales[['gr_Cereales_percapita', 'cal_Cereales_percapita']].describe()
df_aggregated_carnes[['gr_Carnes_percapita', 'cal_Carnes_percapita']].describe()


In [ ]:
# Frutas
df_merged['gr_Frutas'] = np.where(df_merged['grupos_2006'] == 8, df_merged['gr_dia'], 0)
df_merged['cal_Frutas'] = np.where(df_merged['grupos_2006'] == 8, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de frutas por hogar
df_aggregated_frutas = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Frutas': 'sum',
    'cal_Frutas': 'sum'
})

# Calcular per cápita
df_aggregated_frutas['gr_Frutas_percapita'] = df_aggregated_frutas['gr_Frutas'] / df_merged['nper_hogar']
df_aggregated_frutas['cal_Frutas_percapita'] = df_aggregated_frutas['cal_Frutas'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Frutas', 'cal_Frutas'], inplace=True)


In [ ]:
# Verduras
df_merged['gr_Vegetales'] = np.where(df_merged['grupos_2006'] == 9, df_merged['gr_dia'], 0)
df_merged['cal_Vegetales'] = np.where(df_merged['grupos_2006'] == 9, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de verduras por hogar
df_aggregated_vegetales = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Vegetales': 'sum',
    'cal_Vegetales': 'sum'
})

# Calcular per cápita
df_aggregated_vegetales['gr_Vegetales_percapita'] = df_aggregated_vegetales['gr_Vegetales'] / df_merged['nper_hogar']
df_aggregated_vegetales['cal_Vegetales_percapita'] = df_aggregated_vegetales['cal_Vegetales'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Vegetales', 'cal_Vegetales'], inplace=True)


In [ ]:
# Papa y Boniatos
df_merged['gr_Papa'] = np.where(df_merged['grupos_2006'] == 10, df_merged['gr_dia'], 0)
df_merged['cal_Papa'] = np.where(df_merged['grupos_2006'] == 10, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de papa por hogar
df_aggregated_papa = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Papa': 'sum',
    'cal_Papa': 'sum'
})

# Calcular per cápita
df_aggregated_papa['gr_Papa_percapita'] = df_aggregated_papa['gr_Papa'] / df_merged['nper_hogar']
df_aggregated_papa['cal_Papa_percapita'] = df_aggregated_papa['cal_Papa'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Papa', 'cal_Papa'], inplace=True)


In [ ]:
# Legumbres
df_merged['gr_Leguminosa'] = np.where(df_merged['grupos_2006'] == 11, df_merged['gr_dia'], 0)
df_merged['cal_Leguminosa'] = np.where(df_merged['grupos_2006'] == 11, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de legumbres por hogar
df_aggregated_leguminosa = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Leguminosa': 'sum',
    'cal_Leguminosa': 'sum'
})

# Calcular per cápita
df_aggregated_leguminosa['gr_Leguminosa_percapita'] = df_aggregated_leguminosa['gr_Leguminosa'] / df_merged['nper_hogar']
df_aggregated_leguminosa['cal_Leguminosa_percapita'] = df_aggregated_leguminosa['cal_Leguminosa'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Leguminosa', 'cal_Leguminosa'], inplace=True)


In [ ]:
# Dulces
df_merged['gr_Dulces'] = np.where(df_merged['grupos_2006'] == 12, df_merged['gr_dia'], 0)
df_merged['cal_Dulces'] = np.where(df_merged['grupos_2006'] == 12, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de dulces por hogar
df_aggregated_dulces = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Dulces': 'sum',
    'cal_Dulces': 'sum'
})

# Calcular per cápita
df_aggregated_dulces['gr_Dulces_percapita'] = df_aggregated_dulces['gr_Dulces'] / df_merged['nper_hogar']
df_aggregated_dulces['cal_Dulces_percapita'] = df_aggregated_dulces['cal_Dulces'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Dulces', 'cal_Dulces'], inplace=True)


In [ ]:
# Caldos y sopas concentradas
df_merged['gr_Caldos'] = np.where(df_merged['grupos_2006'] == 13, df_merged['gr_dia'], 0)
df_merged['cal_Caldos'] = np.where(df_merged['grupos_2006'] == 13, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de caldos por hogar
df_aggregated_caldos = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Caldos': 'sum',
    'cal_Caldos': 'sum'
})

# Calcular per cápita
df_aggregated_caldos['gr_Caldos_percapita'] = df_aggregated_caldos['gr_Caldos'] / df_merged['nper_hogar']
df_aggregated_caldos['cal_Caldos_percapita'] = df_aggregated_caldos['cal_Caldos'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Caldos', 'cal_Caldos'], inplace=True)


In [ ]:
# Mostrar frecuencia de per cápita para grupos de alimentos
df_aggregated_frutas[['gr_Frutas_percapita', 'cal_Frutas_percapita']].describe()
df_aggregated_vegetales[['gr_Vegetales_percapita', 'cal_Vegetales_percapita']].describe()
df_aggregated_papa[['gr_Papa_percapita', 'cal_Papa_percapita']].describe()
df_aggregated_leguminosa[['gr_Leguminosa_percapita', 'cal_Leguminosa_percapita']].describe()
df_aggregated_dulces[['gr_Dulces_percapita', 'cal_Dulces_percapita']].describe()


In [ ]:
# Otros alimentos
df_merged['gr_Otro'] = np.where(df_merged['grupos_2006'] == 16, df_merged['gr_dia'], 0)
df_merged['cal_Otro'] = np.where(df_merged['grupos_2006'] == 16, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de otros alimentos por hogar
df_aggregated_otro = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Otro': 'sum',
    'cal_Otro': 'sum'
})

# Calcular per cápita
df_aggregated_otro['gr_Otro_percapita'] = df_aggregated_otro['gr_Otro'] / df_merged['nper_hogar']
df_aggregated_otro['cal_Otro_percapita'] = df_aggregated_otro['cal_Otro'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Otro', 'cal_Otro'], inplace=True)


In [ ]:
# Agua, refrescos y jugos
df_merged['gr_Bebida'] = np.where(df_merged['grupos_2006'] == 17, df_merged['gr_dia'], 0)
df_merged['cal_Bebida'] = np.where(df_merged['grupos_2006'] == 17, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de bebidas por hogar
df_aggregated_bebida = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Bebida': 'sum',
    'cal_Bebida': 'sum'
})

# Calcular per cápita
df_aggregated_bebida['gr_Bebida_percapita'] = df_aggregated_bebida['gr_Bebida'] / df_merged['nper_hogar']
df_aggregated_bebida['cal_Bebida_percapita'] = df_aggregated_bebida['cal_Bebida'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Bebida', 'cal_Bebida'], inplace=True)


In [ ]:
# Bebidas alcohólicas
df_merged['gr_Alcohol'] = np.where(df_merged['grupos_2006'] == 18, df_merged['gr_dia'], 0)
df_merged['cal_Alcohol'] = np.where(df_merged['grupos_2006'] == 18, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de alcohol por hogar
df_aggregated_alcohol = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Alcohol': 'sum',
    'cal_Alcohol': 'sum'
})

# Calcular per cápita
df_aggregated_alcohol['gr_Alcohol_percapita'] = df_aggregated_alcohol['gr_Alcohol'] / df_merged['nper_hogar']
df_aggregated_alcohol['cal_Alcohol_percapita'] = df_aggregated_alcohol['cal_Alcohol'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Alcohol', 'cal_Alcohol'], inplace=True)


In [ ]:
# Comidas y bebidas fuera del hogar
df_merged['gr_RH'] = np.where(df_merged['grupos_2006'] == 19, df_merged['gr_dia'], 0)
df_merged['cal_RH'] = np.where(df_merged['grupos_2006'] == 19, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de comidas fuera del hogar por hogar
df_aggregated_rh = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_RH': 'sum',
    'cal_RH': 'sum'
})

# Calcular per cápita
df_aggregated_rh['gr_RH_percapita'] = df_aggregated_rh['gr_RH'] / df_merged['nper_hogar']
df_aggregated_rh['cal_RH_percapita'] = df_aggregated_rh['cal_RH'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_RH', 'cal_RH'], inplace=True)


In [ ]:
# Programas Sociales
df_merged['gr_Progsociales'] = np.where(df_merged['grupos_2006'] == 20, df_merged['gr_dia'], 0)
df_merged['cal_Progsociales'] = np.where(df_merged['grupos_2006'] == 20, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de programas sociales por hogar
df_aggregated_progsociales = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_Progsociales': 'sum',
    'cal_Progsociales': 'sum'
})

# Calcular per cápita
df_aggregated_progsociales['gr_Progsociales_percapita'] = df_aggregated_progsociales['gr_Progsociales'] / df_merged['nper_hogar']
df_aggregated_progsociales['cal_Progsociales_percapita'] = df_aggregated_progsociales['cal_Progsociales'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_Progsociales', 'cal_Progsociales'], inplace=True)


In [ ]:
# Ultraprocesados
df_merged['gr_ULT'] = np.where(df_merged['Ultra_mps'] == 1, df_merged['gr_dia'], 0)
df_merged['cal_ULT'] = np.where(df_merged['Ultra_mps'] == 1, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de ultraprocesados por hogar
df_aggregated_ultra = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_ULT': 'sum',
    'cal_ULT': 'sum'
})

# Calcular per cápita
df_aggregated_ultra['gr_ultra_percapita'] = df_aggregated_ultra['gr_ULT'] / df_merged['nper_hogar']
df_aggregated_ultra['cal_ultra_percapita'] = df_aggregated_ultra['cal_ULT'] / df_merged['nper_hogar']

# Eliminar variables temporales
df_merged.drop(columns=['gr_ULT', 'cal_ULT'], inplace=True)

# Calcular proporción de calorías ultraprocesadas
df_aggregated_ultra['prop_ultra_percapita'] = df_aggregated_ultra['cal_ultra_percapita'] * 100 / df_merged['Cal_percapita']


In [ ]:
# Resumen de calorías per cápita y otras variables
summary = df_merged.groupby('REGION').agg({
    'Cal_percapita': ['mean', 'median'],
    'adec_proteinas': ['mean', 'median'],
    'adec_hidratos': ['mean', 'median'],
    'adec_grasas': ['mean', 'median'],
    'adec_AGS': ['mean', 'median'],
    'sal_gr': ['mean', 'median'],
    'sodio_mg': ['mean', 'median'],
    'fibra_gr': ['mean', 'median'],
    'hierro_percapita': ['mean', 'median'],
    'AF_percapita': ['mean', 'median'],
    'prop_azucar_libre': ['mean', 'median'],
    'K_percapita': ['mean', 'median'],
    'gr_vegyfruta_percapita': ['mean', 'median'],
    'gr_Pescado_percapita': ['mean', 'median']
})
summary
